In [ ]:
#| hide
#| eval: false
! [ -e /content ] && pip install -Uqq xcube  # upgrade xcube on colab

In [ ]:
#| export
from fastai.torch_imports import *
from fastai.torch_core import *
from fastai.callback.core import *
from fastcore.all import *
from xcube.imports import *
from xcube.metrics import *

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp text.callbacks

# XML Callbacks

> General purpose callbacks needed for XML TextLearner

When the target is 1 we want the input to be close to 1 to incur low loss. So when the target is 1 we need to find out how much we need to boost the pred such that its sigmoid is close to 1.

In [ ]:
#| export
class LabelForcing(Callback):
    def __init__(self, end_epoch):
        self.end_epoch = end_epoch
    
    @torch.no_grad()
    def after_pred(self):
        if self.training and self.epoch <= self.end_epoch:
            pred, attn, wgts = self.learn.pred
            pos = Tensor(self.y) == 1
            # pred[pos] += 3*pred.std()
            # pred[~pos] -= 3*pred.std()
            attn[pos] += attn[pos].std(dim=-1).unsqueeze(-1)
            attn[~pos] -= attn[~pos].std(dim=-1).unsqueeze(-1)

In [ ]:
#| hide
#| eval: false
# xb, yb = dls_clas.one_batch()
# pred, attn, wgts = learn.model.to(0)(xb)
# L(pred, yb, attn, wgts).map(Self.shape())
# pos = Tensor(yb) == 1
# before_pos, before_neg = attn[pos].sum(-1).mean(), attn[~pos].sum(-1).mean()
# after_pos, after_neg = attn[pos].sum(-1).mean(), attn[~pos].sum(-1).mean()
# assert before_pos < after_pos and before_neg > after_neg

In [ ]:
lf = LabelForcing(1) 
# cbs = L( [cbs]+ [lf])
# cbs

In [ ]:
#| export
from fastai.callback.progress import *
from fastai.learner import Recorder

In [ ]:
#| export
class _FakeLearner: 
    def to_detach(self,b,cpu=True,gather=True):
        return to_detach(b,cpu,gather)

In [ ]:
#| export
class RunningCallback(Callback):
    order=ProgressCallback.order-1
    
    def __init__(self, mets):
        self.mets = mets
        
    def before_train(self): 
        self.val_cyclit = iter(self.dls.valid) #itertools.cycle(self.dls.valid) 
        self._fake_l = _FakeLearner()
        self.mets.map(Self.reset())
        self.i = 0
        
    def _batch_gen():
        
    
    def before_batch(self):
        self._btch = next(self.val_cyclit)
    
    def after_batch(self):
        if not self.training: return
        self.model.eval()
        self.learn.training=False
        xb, yb = self._btch
        print(self.i, end=' ')
        self.i += 1k
        self._fake_l.yb = (yb,)
        self._fake_l.pred, *_ = self.model(xb) 
        self._fake_l.loss = Tensor(self.loss_func(self._fake_l.pred, yb))
        for met in self.mets: met.accumulate(self._fake_l)
        self.model.train()
        self.learn.training=True
    
    def after_train(self):
        pdb.set_trace()
        if hasattr(self, 'val_cyclit'): delattr(self, 'val_cyclit')

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()